<font style="font-size:28px;" align="left"><b> Multiplication of 2 numbers using qiskit </b></font>
<br>
Prepared by Hussein Shiri
<br><br>

### Introduction

In this notebook, a solution for multiplying 2 numbers using qiskit is proposed.

The main methodology consists of 2 steps:

    1. Implementing a function that adds 2 numbers using qiskit
    
    2. Multiplying 2 numbers using repetitive addition

### Importing before we begin

In [1]:
from random import randrange

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute

### Generating two random numbers <= 23

In [2]:
# When i ran my multiply(A, B) function, whenever i enter a number > 23 the kernel dies
# So our random number will be <= 23
A = randrange(0, 24)

B = randrange(0, 24)

print(f"A = {A} and B = {B}")

A = 7 and B = 8


### Function that changes from integer to binary

In [3]:
def binary(c):
    '''
    Converts an integer c to a binary string of at most 10 digits
    So the maximum number the function can take as input is 1023
    Returns the binary number as a string
    '''
    # first checks if the number c == 0
    if c == 0:
        return '0'
    
    # the following code is executed if c != 0
    # if action is True a '1' is added at the end of the string c_binary after finishing
    # example: '01001' --> '010011'
    c_binary = ''
    action = False
    if c%2 != 0:
        c = c-1
        action = True
    
    # finds the binary representation of the number c
    for j in [9, 8, 7, 6, 5, 4, 3, 2, 1]:
        to_subtract = 1
        
        # finds 2^j
        for k in range(j):
            to_subtract *= 2
        
        if c-to_subtract >= 0:
            c = c-(to_subtract)
            c_binary += '1'
        else:
            c_binary += '0'
    
    # adds '1' at the end of the string if action == True
    if action == True:
        c_binary += '1'
    else:
        c_binary += '0'

    j = 0
    # deletes any zero's at the beginning of the string c_binary
    # example '0010110' --> '10110'
    c_binary_final = ''
    while c_binary[j] == '0':
        j = j + 1
    for k in range(j, len(c_binary)):
        c_binary_final += c_binary[k]
    c_binary = c_binary_final
    
    return c_binary

A_binary = binary(A)
B_binary = binary(B)
print(f"In binary: A = {A_binary} and B = {B_binary}")

In binary: A = 111 and B = 1000


### Implementing an adder function

In [4]:
def adder(l, l2):
    '''
    Adds 2 numbers l and l2
    Returns the counts
    example: l = '10', l2 = '01' returns {'011': number_of_shots}
    example:
    a = 01, b= 01
    a+b = 10
    a+b = 0 1
        + 0 1
        ------
          0 0
        + 1   (1 is the ancilla bit or remaining bit)
        ------
        = 10
    '''
    
    if len(l) > len(l2):
        n = len(l)
    else:
        n = len(l2)
    
    # reserves the number of classical and quantum registers required
    # 2n quantum registers to store the inputs l --> a and l2 --> b
    # n+1 quantum registers to store the ancilla bits --> c
    # n+1 quantum registers to store the output number --> d
    # n+1 classical registers to measure the output number(register d) --> cl
    a = QuantumRegister(n)
    b = QuantumRegister(n)
    c = QuantumRegister(n + 1)
    d = QuantumRegister(n + 1)
    cl = ClassicalRegister(n + 1)
    # puts all the registers together in 1 circuit
    qc = QuantumCircuit(a, b, c, d, cl)
    
    # prepare the input numbers using qubits(registers a and b)
    for i in range(len(l)):
        if l[i] == '1':
            qc.x(a[len(l) - (i+1)])
    for i in range(len(l2)):
        if l2[i] == '1':
            qc.x(b[len(l2) - (i+1)])
    
    # start the operation of addition
    for i in range(n):
        # find the output bits
        # example: if a=1 and b=1 and ancilla(remaing) bit c=1 then d = 1 with an extra remaining bit
        # example: if a=1 and b=1 and ancilla(remaing) bit c=0 then d = 0 with an extra remaining bit
        qc.cx(c[i], d[i])
        qc.cx(a[i], d[i])
        qc.cx(b[i], d[i])
        
        # find the ancilla bits 
        qc.ccx(a[i], b[i], c[i+1])
        qc.ccx(a[i], c[i], c[i+1])
        qc.ccx(b[i], c[i], c[i+1])
    # find the last output bit
    # an extra ouput bit might be needed because there might be an additional remaining bit from previous addtition
    qc.cx(c[i+1], d[i+1])
    
    # Now addition finished so measure the circuit
    # Note: range(n+1) and not range(n) due to the extra output bit
    for i in range(n+1):
        qc.measure(d[i], cl[i])
    
    # to draw the circuit the below code can be uncommented
    # display(qc.draw(output = 'mpl', reverse_bits = True))
    
    # execute the circuit
    num_shots = 2
    job = execute(qc, Aer.get_backend('qasm_simulator'), shots=num_shots)
    
    #Get results of program
    job_stats = job.result().get_counts()
    
    #return the results(counts)
    return job_stats

print(adder(A_binary, B_binary))

{'01111': 2}


### Implementing a function that multiplies 2 numbers

In [5]:
def multiply(A, B):
    '''
    Takes 2 integers A and B
    Uses repetitive addition to find final_answer
    Returns the multiplication of A and B
    example:
    A = 3
    B = 5
    answer = 0
    Repeat A times ==> Repeat 3 times
    1st time: answer = answer + B = 0 + 5 = 5
    2nd time: answer = answer + B = 5 + 5 = 10
    3rd time: answer = answer + B = 10 + 5 = 15
    return answer
    '''
    
    # transform the values from integer to binary
    first = binary(A)
    second = binary(B)
    
    answer = '0'
    if int(first, 2) != 0 and int(second, 2) != 0: # int(first, 2): transforms from binary to integer
        for _ in range(int(first, 2)): # repeat the addition A times
            
            preanswer = ''
            # start the addition process
            answer = adder(answer, second)
            
            # takeoff any extra zero's at the beginning of the string
            # the end string is put in the variable answer
            # example: '01010' --> '1010'
            for key in answer: answer = key
            j = 0
            while key[j] == '0':
                j = j + 1
            for k in range(j, len(key)):
                preanswer += key[k]
            answer = preanswer
    
    # prints the multiplication result as binary
    print(answer)
    # changes final_answer from binary to integer
    final_answer = int(answer, 2)
    # prints the multiplication result as integer
    print(final_answer)

multiply(A, B)

111000
56


# Extra

##### Second method main methodology and how to perform swap

The main methodology of the second method focuses on changing how the adder function works.

Instead of having n+1 'd' quantum registers, we have only 1 'd' quantum register. And the output number is stored
in the n+1 'b' registers.

In the 'b' registers the first n have the input number stored in them. After finding the first digit of the output 
number and storing it in 'd', we use swap to put the value in the 'd' register in the first 'b' register and then we reset the value of the 'd' register to 0. And we repeat for the second, third,... values.

##### Example

a0 = 1, b0 = 0, c0 = 0, d0 = 0

d0 becomes 1

==> a0 = 1, b0 = 0, c0= 0, d0 = 1

then put the value of d0 in b0 and reset d0

==> a0 = 1, b0 = 1, c0 = 0, d0 = 0

And continue in the same process for a1, b1, c1, d0

# Optimizing the function adder so that it uses less qubits but more gates

In [6]:
def adder_optimized(l, l2):
    '''
    Adds 2 numbers l and l2
    Returns the counts
    example: l = '10', l2 = '01' returns {'011': number_of_shots}
    example:
    a = 01, b= 01
    a+b = 10
    a+b = 0 1
        + 0 1
        ------
          0 0
        + 1   (1 is the ancilla bit or remaining bit)
        ------
        = 10
    '''
    
    if len(l) > len(l2):
        n = len(l)
    else:
        n = len(l2)
    
    # reserves the number of classical and quantum registers required
    # 2n+1 quantum registers to store the inputs l --> a and l2 --> b, the extra 1 for the extra output bit
    # n+1 quantum registers to store the ancilla bits --> c
    # 1 quantum registers to store the output number --> d
    # later the value of d will be stored in the b's registers using swap
    # n+1 classical registers to measure the output number(register d) --> cl
    a = QuantumRegister(n)
    b = QuantumRegister(n + 1)
    c = QuantumRegister(n + 1)
    d = QuantumRegister(1)
    cl = ClassicalRegister(n + 1)
    # puts all the registers together in one circuit
    qc = QuantumCircuit(a, b, c, d, cl)
    
    # prepare the input numbers using qubits(registers a and b)
    for i in range(len(l)):
        if l[i] == '1':
            qc.x(a[len(l) - (i+1)])
    for i in range(len(l2)):
        if l2[i] == '1':
            qc.x(b[len(l2) - (i+1)])
    
    for i in range(n):
        # for the below code see the above provided example and the previous adder() function
        qc.cx(c[i], d[0])
        qc.cx(a[i], d[0])
        qc.cx(b[i], d[0])
        
        qc.ccx(a[i], b[i], c[i+1])
        qc.ccx(a[i], c[i], c[i+1])
        qc.ccx(b[i], c[i], c[i+1])
        
        qc.cx(b[i], d[0])
        qc.cx(d[0], b[i])
        qc.cx(b[i], d[0])
        qc.reset(d[0])
    qc.cx(c[i+1], b[i+1])
    
    
    for i in range(n+1):
        qc.measure(b[i], cl[i])
    
    #execute the circuit
    num_shots = 2
    job = execute(qc, Aer.get_backend('qasm_simulator'), shots=num_shots)
    
    #Get results of program
    job_stats = job.result().get_counts()
    
    #return the results
    return job_stats

print(adder_optimized(A_binary, B_binary))

{'01111': 2}


### Redoing multiplication

##### Before that a small comparison

The adder() function required 5n+1 qubits but less gates.

The adder_optimized() function required 4n+1 qubits but more gates.

Found the adder_optimized() function to be a lot slower. That is due to the swap operation. Because if two qubits 
are not near each other, swapping them would require even more additional gates.

So due to this slow speed numbers A = 7, and B = 5 are provided below.

In [7]:
A = 7
B = 5
# the function multiply_optimized is the same as the function multiply but with only some name changes
# multiply --> multiply_optimized
# adder --> adder_optimized
def multiply_optimized(A, B):
    
    first = binary(A)
    second = binary(B)
    
    answer = '0'
    if int(first, 2) != 0 and int(second, 2) != 0:
        
        for _ in range(int(first, 2)):
            
            preanswer = ''
            answer = adder_optimized(answer, second)
            
            for key in answer: answer = key
            j = 0
            while key[j] == '0':
                j = j + 1
            for k in range(j, len(key)):
                preanswer += key[k]
            answer = preanswer
            
    print(answer)
    final_answer = int(answer, 2)
    print(final_answer)

multiply_optimized(A, B)

100011
35
